In [1]:
cd ..

/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling


/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# make sure you're logged in with \`huggingface-cli login\`
# import torch
# import argparse
import os
import torch
from torchvision.utils import make_grid, save_image
import numpy as np
# from PIL import Image
import matplotlib.pyplot as plt
# import os
import lpips
from PIL import Image
import hydra
from hydra import initialize, compose
from omegaconf import DictConfig, OmegaConf
import torchvision.transforms as transforms 
from torchvision.datasets import VisionDataset
from torch.utils.data import DataLoader
from typing import Callable, Optional
# import glob
import torch.nn as nn

from datasets import build_loader

%load_ext autoreload
%autoreload 2

/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load config file
# with initialize(version_base=None, config_path="../_configs"):
#     cfg = compose(config_name='ddrmpp.yaml')

# # Define upsample operation
upsample = nn.Upsample(scale_factor=2, mode='nearest') 
# downsample = nn.AvgPool2d(2, 2)
# downsample = F.interpolate(scale_factor=0.5, mode='nearest')

# # Build loader
# loader = build_loader(cfg)
lpips_ = lpips.LPIPS(net='alex').cuda()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth


## REDDIF

In [1]:
# algo = 'reddiff'
# algo = 'rsd_stable'
algo = 'rsd_stable'
max_img_id = '00003'

img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/_exp/output/{algo}'
# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/_exp/output/{algo}/inp_large_box'
img_path_input = '/home/nicolas/dataset/imagenet-root/imagenet/val/n01484850/'
coeff = 0.0
import os
import torch
from PIL import Image

dino = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
dino = dino.eval()
# Calculate diversity
cnt = 0
mean_psnr = 0
mean_lpips = 0
for item in os.listdir(img_path_input):
    if item.split('.')[0] == max_img_id:
        # Check if the item is a directory (folder)
        img_psld_torch_list = []    
        for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
            # if img_id.split('_')[2] == "grid.png":
            #     continue
            img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
            img = Image.open(img_id_path).convert('RGB')

            img_psld_torch = transforms.ToTensor()(img)
            img_psld_torch_list.append(img_psld_torch)
                # Transform to tensor the list
        img_psld_torch_list = torch.stack(img_psld_torch_list)
        # print(img_psld_torch_list.shape)
        # Compute similarity
        total_pair_wise_sim = 0.
        xo_list = dino(img_psld_torch_list)
        # del dino
        xo_list /= xo_list.norm(dim=-1, keepdim=True)
        # calculate the cosine similarity
        sim = (xo_list @ xo_list.T)
        # set the diagonal to be 0
        sim = sim - torch.diag(sim.diag())
        total_pair_wise_sim += sim.sum() / (sim.shape[0] * (sim.shape[0] - 1))
        torch.clear_autocast_cache()
        # print(total_pair_wise_sim)
                            
print(total_pair_wise_sim)

Using cache found in /home/nicolas/.cache/torch/hub/facebookresearch_dino_main
/home/nicolas/anaconda3/envs/stable-dif/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/_exp/output/rsd_stable_repulsion_gamma0/00003'

In [21]:
# algo = 'reddiff'
algo = 'rsd_stable'
gamma = 0
# algo = f'rsd_stable_repulsion_gamma{gamma}'
max_img_id = 'ILSVRC2012_val_00000329'

# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}'
img_path = '/home/nicolas/server_nvidia/PSLD/stable-diffusion/outputs/psld-samples-bip/samples'
# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/_exp/output/{algo}/inp_large_box'
img_path_input = '/home/nicolas/dataset/imagenet-root/imagenet/val/n01514668/'
# img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/datasets/samples'
coeff = 0.0

if algo == 'rsd_stable' or algo == f'rsd_stable_repulsion_gamma{gamma}':
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path):
        # print(item)
        if item.split('.')[0] == max_img_id:
            # print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item + '.JPEG')
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            # img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            img_true_torch = transforms.Resize((256, 256))(img_true_torch)
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                print(img_id)
                # if img_id.split('_')[2] == "grid.png":
                #     continue         
                img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                img = Image.open(img_id_path).convert('RGB')

                img_psld_torch = transforms.ToTensor()(img)
                img_psld_torch = transforms.Resize((256, 256))(img_psld_torch)
                # print(img_psld_torch.shape)
                # img_psld_torch= nn.functional.interpolate(img_psld_torch.unsqueeze(0), size=(256, 256)).squeeze()

                mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                psnr = 10 * torch.log10(1 / (mse + 1e-10))
                print(psnr)
                mean_psnr = mean_psnr + psnr

                lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                print(lpips_val)
                mean_lpips = mean_lpips + lpips_val

                cnt = cnt + 1
else:
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path_input):
        if item.split('.')[0] == max_img_id:
            # print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item)
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            # img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            img_true_torch = transforms.Resize((256, 256))(img_true_torch)
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                if img_id.split('_')[2] == "grid.png":
                    continue
                img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                img = Image.open(img_id_path).convert('RGB')

                img = transforms.Resize((256, 256))(img)
                img_psld_torch = transforms.ToTensor()(img)
                

                mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                psnr = 10 * torch.log10(1 / (mse + 1e-10))
                print(psnr)
                mean_psnr = mean_psnr + psnr

                lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                print(lpips_val)
                mean_lpips = mean_lpips + lpips_val

                cnt = cnt + 1


print(mean_psnr/cnt)
print(mean_lpips/cnt)

00009.png
tensor(25.1942)
tensor([[[[0.0535]]]], device='cuda:0', grad_fn=<AddBackward0>)
00008.png
tensor(25.3296)
tensor([[[[0.0657]]]], device='cuda:0', grad_fn=<AddBackward0>)
00010.png
tensor(25.8126)
tensor([[[[0.0508]]]], device='cuda:0', grad_fn=<AddBackward0>)
tensor(25.4455)
tensor([[[[0.0567]]]], device='cuda:0', grad_fn=<DivBackward0>)


## Large scale

In [33]:
algo = 'rsd_stable'
# algo = 'psld'
gamma = 0


# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}'
if algo == 'rsd_stable':
    img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/rsd_stable/inp_free_imagenet'
elif algo == 'psld':
    img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/PSLD/output_bip_large_imagenet/samples'
# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/_exp/output/{algo}/inp_large_box'
img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/input/IMAGENET'
# img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/datasets/samples'
coeff = 0.0

if algo == 'rsd_stable' or algo == f'rsd_stable_repulsion_gamma{gamma}':
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path):
        print(item)

        img_true_path = os.path.join(img_path_input, item + '.png')
        # print(item)
        img_true = Image.open(img_true_path).convert('RGB')
        img_true_torch = transforms.ToTensor()(img_true)
        img_true_torch = transforms.Resize((512, 512))(img_true_torch)
        
        psnr_list = []
        lpips_list = []
        psnr_name_list = []
        lpips_name_list = []

        for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
            # print(img_id)
            if algo == 'psld':
                if len(img_id.split('.')[0].split('_')) == 3:
                    # if img_id.split('.')[0].split('_')[1] > "00004":
                    continue    
            elif algo == 'rsd_stable':
                if img_id.split('_')[2] == "grid.png":
                    continue
            img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
            img = Image.open(img_id_path).convert('RGB')

            img_psld_torch = transforms.ToTensor()(img)
            # img_psld_torch = transforms.Resize((512, 256))(img_psld_torch)
            # print(img_psld_torch.shape)
            # img_psld_torch= nn.functional.interpolate(img_psld_torch.unsqueeze(0), size=(256, 256)).squeeze()

            mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
            psnr = 10 * torch.log10(1 / (mse + 1e-10))
            psnr_list.append(psnr)
            psnr_name_list.append(img_id)
            # print(psnr)

            lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
            lpips_list.append(lpips_val)
            lpips_name_list.append(img_id)

            # cnt = cnt + 1
        
        # print(f"Mean PSNR: {torch.max(torch.stack(psnr_list))}", psnr_name_list[torch.argmax(torch.stack(psnr_list))])
        mean_psnr = mean_psnr + torch.max(torch.stack(psnr_list))
        # print(f"Mean LPIPS: {torch.min(torch.stack(lpips_list))}", lpips_name_list[torch.argmin(torch.stack(lpips_list))])
        
        # # Open the image with minimum lpips
        # img = Image.open(os.path.join(img_path, item.split('.')[0], lpips_name_list[torch.argmin(torch.stack(lpips_list))]))
        # img = transforms.ToTensor()(img)
        # # Save image
        # save_image(img, f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/rsd_stable/FID/{item}.png')
        
      

        mean_lpips = mean_lpips + torch.min(torch.stack(lpips_list))
        cnt = cnt + 1


print(mean_psnr/cnt)
print(mean_lpips/cnt)
print(cnt)

n02098286
n02137549
n04367480
n04026417
n02066245
n04501370
n04118538
n03670208
n02823750
n03781244
n01828970
n02988304
n04265275
n03028079
n02488291
n02106030
n03218198
n04579432
n02098413
n01984695
n07734744
n03983396
n03785016
n02109047
n01756291
n03476684
n04399382
n01698640
n04429376
n04591713
n02107683
n02363005
n03710721
n01440764
n07695742
n04336792
n02106166
n07920052
n02676566
n02978881
n04590129
n01695060
n03134739
n02794156
n03000684
n02107908
n03733805
n01514668
n02786058
n03866082
n02788148
n07760859
n02091831
n01749939
n02268853
n02093256
n01855032
n01768244
n02112350
n04557648
n02412080
n01990800
n03676483
n03291819
n02089078
n07716358
n04332243
n02793495
n01592084
n04204238
n01744401
n07802026
n02814860
n02356798
n03887697
n02111889
n02086646
n02514041
n02835271
n02640242
n02090622
tensor(26.7735)
tensor(0.0751, device='cuda:0', grad_fn=<DivBackward0>)
81


In [34]:
# algo = 'reddiff'
algo = 'psld'
gamma = 0


# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/{algo}'
if algo == 'rsd_stable':
    img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/rsd_stable/inp_free_imagenet'
elif algo == 'psld':
    img_path = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/output/PSLD/output_bip_large_imagenet/samples'
# img_path = f'/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/_exp/output/{algo}/inp_large_box'
img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/_exp/input/IMAGENET'
# img_path_input = '/home/nicolas/Repulsive-score-distillation-RSD-/constrained_sampling/datasets/samples'
coeff = 0.0


# Calculate FID score
cnt = 0
mean_psnr = 0
mean_lpips = 0
for item in os.listdir(img_path):
    print(item)

    img_true_path = os.path.join(img_path_input, item + '.png')
    # print(item)
    img_true = Image.open(img_true_path).convert('RGB')
    img_true_torch = transforms.ToTensor()(img_true)
    img_true_torch = transforms.Resize((512, 512))(img_true_torch)
    
    psnr_list = []
    lpips_list = []
    psnr_name_list = []
    lpips_name_list = []

    # Check if list is not empty
    if os.listdir(os.path.join(img_path, item.split('.')[0])) == []:
        continue

    for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
        # print(img_id)
        if algo == 'psld':
            if len(img_id.split('.')[0].split('_')) == 3:
                # if img_id.split('.')[0].split('_')[1] > "00004":
                continue    
        elif algo == 'rsd_stable':
            if img_id.split('_')[2] == "grid.png":
                continue
        img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
        img = Image.open(img_id_path).convert('RGB')

        img_psld_torch = transforms.ToTensor()(img)
        # img_psld_torch = transforms.Resize((512, 256))(img_psld_torch)
        # print(img_psld_torch.shape)
        # img_psld_torch= nn.functional.interpolate(img_psld_torch.unsqueeze(0), size=(256, 256)).squeeze()

        mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
        psnr = 10 * torch.log10(1 / (mse + 1e-10))
        psnr_list.append(psnr)
        psnr_name_list.append(img_id)
        # print(psnr)

        lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
        lpips_list.append(lpips_val)
        lpips_name_list.append(img_id)

        # cnt = cnt + 1
    
    # print(f"Mean PSNR: {torch.max(torch.stack(psnr_list))}", psnr_name_list[torch.argmax(torch.stack(psnr_list))])
    mean_psnr = mean_psnr + torch.max(torch.stack(psnr_list))
    # print(f"Mean LPIPS: {torch.min(torch.stack(lpips_list))}", lpips_name_list[torch.argmin(torch.stack(lpips_list))])
    mean_lpips = mean_lpips + torch.min(torch.stack(lpips_list))
    cnt = cnt + 1


print(mean_psnr/cnt)
print(mean_lpips/cnt)
print(cnt)

n02098286
n02137549
n04367480
n04026417
n02066245
n04501370
n04118538
n03670208
n02823750
n03781244
n01828970
n02988304
n04265275
n03028079
n02488291
n02106030
n03218198
n04579432
n02098413
n01984695
n07734744
n03983396
n03785016
n02109047
n01756291
n03476684
n04399382
n01698640
n04429376
n04591713
n02107683
n02363005
n03710721
n01440764
n07695742
n04336792
n02106166
n07920052
n02676566
n02978881
n04590129
n01695060
n03134739
n02794156
n03000684
n02107908
n03733805
n01514668
n02786058
n03866082
n02788148
n07760859
n02091831
n01749939
n02268853
n02093256
n01855032
n01768244
n02112350
n04557648
n02412080
n01990800
n03676483
n02096294
n03291819
n02089078
n07716358
n04332243
n02793495
n01592084
n04204238
n01744401
n07802026
n02814860
n02356798
n03887697
n02111889
n02086646
n02514041
n02835271
n02640242
n02090622
n01677366
n02114548
n03388043
n02509815
n03445924
n03884397
n04033995
n04152593
n03729826
n07745940
tensor(22.5568)
tensor(0.1544, device='cuda:0', grad_fn=<DivBackward0>)
91
